# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [2]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # subsample the data 
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    
    mask = range(num_test, num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]
    
    # Shape of the data that comes in
    print('\n\nShape of raw data from dataset:')
    print('Training set\n{}\n\nLabel set:\n{}\n'
          .format(X_test.shape, y_test.shape))
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    # current shape
    print('Reshaped data to M x N (examples x features):')
    print('Train data shape: ', X_train.shape)
    print('Train labels shape: ', y_train.shape)
    print('Validation data shape: ', X_val.shape)
    print('Validation labels shape: ', y_val.shape)
    print('Test data shape: ', X_test.shape)
    print('Test labels shape: ', y_test.shape)

    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))]).T
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))]).T
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))]).T

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()

print('\n\nShape after adding bias dimension and transforming into columns:')
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)




Shape of raw data from dataset:
Training set
(1000, 32, 32, 3)

Label set:
(1000,)

Reshaped data to M x N (examples x features):
Train data shape:  (48000, 3072)
Train labels shape:  (48000,)
Validation data shape:  (1000, 3072)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3072)
Test labels shape:  (1000,)


Shape after adding bias dimension and transforming into columns:
Train data shape:  (3073, 48000)
Train labels shape:  (48000,)
Validation data shape:  (3073, 1000)
Validation labels shape:  (1000,)
Test data shape:  (3073, 1000)
Test labels shape:  (1000,)


## Softmax Classifier

Your code for this section will all be written inside **cs231n/classifiers/softmax.py**. 


In [4]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(10, 3073) * 0.0001
print('Weight Matrix shape: {}\n'.format(W.shape))
loss, grad = softmax_loss_naive(W, X_train, y_train, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: {}'.format(loss))
print('sanity check: {}'.format(-np.log(0.1)))

Weight Matrix shape: (10, 3073)

loss: 2.3130958731983498
sanity check: 2.3025850929940455


## Inline Question 1:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

**Your answer:** *Fill this in*


In [4]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_train, y_train, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_train, y_train, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: -0.019166698428385587 analytic: -0.014545831731222352, relative error: 0.13706674270030442
numerical: -3.2899021418675285 analytic: -3.835120761498654, relative error: 0.07652166554770509
numerical: -0.921849289925447 analytic: -0.6323791803634247, relative error: 0.18624681962505868
numerical: 0.368799114558449 analytic: 0.1476340328062346, relative error: 0.4282550081860582
numerical: -1.2150419530509993 analytic: -1.0422808776698673, relative error: 0.07653361452334287
numerical: -0.920908368717832 analytic: -0.6445193287989854, relative error: 0.17655816385341383
numerical: 0.7587133501019138 analytic: -0.054521306887924226, relative error: 1.0
numerical: 2.1181559898053592 analytic: 1.602808607317001, relative error: 0.1384983299456562
numerical: -1.699020146417318 analytic: -1.1917029673402793, relative error: 0.17549836463499494
numerical: -0.41247556936685464 analytic: -0.11448713703041859, relative error: 0.5654829624921973


In [5]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_train, y_train, 0.00001)
toc = time.time()
print('naive loss: {} computed in {}'.format(loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_train, y_train, 0.00001)
toc = time.time()
print('vectorized loss: {} computed in {}'.format(loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: {}'.format(np.abs(loss_naive - loss_vectorized)))
print('Gradient difference: {}'.format(grad_difference))

naive loss: 2.3130958747428467 computed in 7.24904465675354
vectorized loss: 2.313095874742853 computed in 0.49280834197998047
Loss difference: 6.217248937900877e-15
Gradient difference: 151.03533510280894


In [41]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 5e-7]
regularization_strengths = [5e4, 1e8]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
folds = 5

print('Training set before fold:\t', X_train.shape)
# Folds 2D array into a 3D array with i layers
X_train_folds = np.array(np.array_split(X_train, folds, axis=1))
y_train_folds = np.array(np.array_split(y_train, folds))
print('Training set after fold:\t', X_train_folds.shape)


for val_fold in range(folds):  # for each fold of data
    
    for lr in learning_rates:
        
        # create a validation set of that fold
        valid_set = X_train_folds[val_fold]
        valid_lbl = y_train_folds[val_fold]

        # remove the validation fold
        train_set = np.delete(X_train_folds, val_fold, axis=0)
        train_lbl = np.delete(y_train_folds, val_fold, axis=0) 

        # reshape into a 2D array
        train_set = train_set.reshape((train_set.shape[1], -1))
        train_lbl = train_lbl.reshape((-1,))

        # train the training subset
        SoftMax_Classifier = LinearClassifier()
        SoftMax_Classifier.train(train_set, train_lbl, learning_rate=lr)

        # test the trained model on the validation set
        predictions = SoftMax_Classifier.predict(X_test)
        
        # add the results to the list
        results[lr] = 
    
    pass
                


pass
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr {} reg {} train accuracy: {} val accuracy: {}'
          .format(lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: {}'.format(best_val))

Training set before fold:	 (3073, 48000)
Training set after fold:	 (5, 3073, 9600)
train set shape:  (4, 3073, 9600)
best validation accuracy achieved during cross-validation: -1


In [12]:
# evaluate on test set
# Evaluate the best svm on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print ('softmax on raw pixels final test set accuracy: {}'.format(test_accuracy, ))

AttributeError: 'NoneType' object has no attribute 'predict'

In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:,:-1] # strip out the bias
w = w.reshape(10, 32, 32, 3)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in xrange(10):
  plt.subplot(2, 5, i + 1)
  
  # Rescale the weights to be between 0 and 255
  wimg = 255.0 * (w[i].squeeze() - w_min) / (w_max - w_min)
  plt.imshow(wimg.astype('uint8'))
  plt.axis('off')
  plt.title(classes[i])